In [ ]:
from random import randint
from collections import Counter
from math import log, sqrt, e as exp
import time

def index_calculus_dlp(p, g, h):
    # === Parameters ===
    B = int(0.5 * exp ** (sqrt((log(p) * log(log(p))) / 2)))
    factor_base = list(primes(B))
    Zmod = Integers(p - 1)  # Ring of integers modulo p-1

    # Faster B-smooth check using trial division
    def is_Bsmooth_fast(b, n):
        factors = []
        for prime in factor_base:
            if prime > b:
                break
            while n % prime == 0:
                factors.append(prime)
                n //= prime
        if n != 1:
            return False, []
        return True, factors

    # Convert factor list to exponent dict
    def factorlist_to_explist(factors):
        return dict(Counter(factors))

    # Find B-smooth congruences: g^t mod p
    def find_congruences_fast():
        congruences = []
        while len(congruences) < len(factor_base):
            t = randint(2, p)
            val = pow(g, t, p)
            smooth, factors = is_Bsmooth_fast(B, val)
            if smooth:
                exponents = factorlist_to_explist(factors)
                congruences.append((exponents, t))
        return congruences

    # Build matrix system over Zmod
    def to_matrix_system(bases, congruences):
        M = Matrix(Zmod, [[exp.get(b, 0) for b in bases] for exp, _ in congruences])
        b = vector(Zmod, [t for _, t in congruences])
        return M, b

    # Evaluate expression with known logs (perform all arithmetic in Zmod)
    def evaluate(factor_dict, dlogs):
        return Zmod(sum(e * dlogs.get(b, 0) for b, e in factor_dict.items()))

    # === Main Algorithm ===
    print(f"Parameters: p={p}, g={g}, h={h}, B={B}")
    congruences = find_congruences_fast()

    print("Solving linear system...")
    M, b = to_matrix_system(factor_base, congruences)
    try:
        exponents = M.solve_right(b)
    except ValueError as e:
        print(f"Failed to solve linear system: {e}")
        return None

    dlogs = dict(zip(factor_base, exponents))

    # Search for B-smooth h*g^t
    print("Searching for B-smooth value of h * g^t...")
    while True:
        t = randint(2, p)
        candidate = (h * pow(g, t, p)) % p
        smooth, factors = is_Bsmooth_fast(B, candidate)
        if smooth:
            break

    log_h = (evaluate(factorlist_to_explist(factors), dlogs) - Zmod(t))
    if pow(g, int(log_h), p) == h:
        print(f"Success: {g}^{log_h} ≡ {h} mod {p}")
        return int(log_h)
    else:
        print("Failed to verify solution.")
        return None

In [ ]:
# 30-bit prime field:
# p = 1004162429
# g = 2
# x = 476110242  # secret discrete log
# h = 375313147
# s=time.time()
# index_calculus_dlp(p=1004162429, g=2, h=375313147)
# e=time.time()
# print(f"{e-s}")

# 40-bit prime field:
# p = 671513713477
# g = 5
# x = 239660902444  # secret discrete log
# h = 263612859886
# s=time.time()
# index_calculus_dlp(p=671513713477, g=5, h=263612859886)
# e=time.time()
# print(f"{e-s}")

# 50-bit prime field:
# p = 720026475387601
# g = 17
# x = 25300657835292  # secret discrete log
# h = 87825293952479
s=time.time()
index_calculus_dlp(p=720026475387601, g=17, h=87825293952479)
e=time.time()
print(f"{e-s}")